In [1]:
import sys
sys.path.append("/root/workspace/LidarObjDetection/mmdetection3d_dona/mmdetection3d")
sys.path.append("/root/workspace/LidarObjDetection/mmdeploy_dona/mmdeploy")

from mmdet3d.apis import LidarDet3DInferencer
import os
import torch
import os
import numpy as np
import time
import open3d as o3d
from mmdet3d.datasets.transforms.loading import (LoadAnnotations3D,
                                                LoadPointsFromFile)
from mmdet3d.models.data_preprocessors.data_preprocessor import \
    Det3DDataPreprocessor
import onnxruntime as ort
from mmdet3d.datasets.transforms.formating import Pack3DDetInputs

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# initialize inferencer
# CHANGE THE center_mode in loca_visualizer accordingly (KITTI -> center_mode = 'lidar_bottom', CUSTOM -> center_mode = whatever)

# inferencer = LidarDet3DInferencer('pointpillars_hv_secfpn_8xb6-160e_kitti-3d-car')
# inferencer = LidarDet3DInferencer('pointpillars_donaset_container-car')
inferencer = LidarDet3DInferencer('pointpillars_cielo_falcon-car')


# inferencer = LidarDet3DInferencer('pointpillars_kittilidar-car') 
# inferencer = LidarDet3DInferencer("pointpillars_hv_secfpn_8xb6-160e_kittilidar-3d-car",scope="mmdet3d")

Loads checkpoint by local backend from path: /root/workspace/LidarObjDetection/mmdetection3d_dona/mmdetection3d/work_dirs/pointpillars_hv_secfpn_8xb6-160e_cielo_falcon-3d-car/iter_5640.pth
02/28 12:58:16 - mmengine - WARNING - Failed to search registry with scope "mmdet3d" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet3d" is a correct scope, or whether the registry is initialized.


/root/workspace/LidarObjDetection/mmdetection3d_dona/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:94: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


# Prediction Tests

In [11]:
# inference
pcl = '../data/cielo_falcon/training/velodyne/'
# pcl = './data/falcon/falcon1.bin'

inputs = dict(points=pcl)
inf_res = inferencer(inputs, show=True,pred_score_thr=0.3, batch_size=4)

[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
[Open3D WARNING] GLFW Error: The GLFW library is not initialized


SystemExit: 0

In [8]:
import time
start = time.time()
inferencer(inputs, show=False,pred_score_thr=0.3,batch_size=4)
end = time.time()
print("Time Taken {} ms".format((end-start)*1000))

Preprocessing took 40.65680503845215 ms
Voxel Encoder took 0.7505416870117188 ms
Middle Encoder/Imagify took 54.91042137145996 ms
Backbone + FPN took 1.1451244354248047 ms
Bbox Head pred took 152.24933624267578 ms

Time Taken 262.5012397766113 ms


In [4]:
from mmdet3d.datasets.transforms.loading import (LoadAnnotations3D,
                                                 LoadPointsFromFile)
from mmdet3d.models.data_preprocessors.data_preprocessor import \
    Det3DDataPreprocessor
from mmdet3d.datasets.transforms.formating import Pack3DDetInputs
import os 

def prepare_input(inputs,batch_size=1):
    model_inputs = {}
    model_inputs["inputs"] = {}
    model_inputs["inputs"]["points"] = []
    max = batch_size

    files = []
    if not os.path.isdir(inputs["points"]):
        files.append(inputs["points"])
    else:
        files = [inputs["points"] + s for s in os.listdir(inputs["points"])]
    for i,sample_path in enumerate(files):
        inp = {}
        inp["lidar_points"] = {}
        inp["lidar_points"]["lidar_path"] = sample_path


        loader = LoadPointsFromFile(coord_type='LIDAR',load_dim=4,use_dim=4)
        packer = Pack3DDetInputs(keys=['points'])

        voxel_size = [0.16, 0.16, 4]
        preprocessor = Det3DDataPreprocessor(
            voxel=True,
            voxel_layer=dict(
                max_num_points=32,  # max_points_per_voxel
                point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1],
                voxel_size=voxel_size,
                max_voxels=(16000, 40000)))

        input = loader(inp)
        input = packer(input)
        # input["inputs"]["points"] = input["inputs"]["points"].cuda()
        model_inputs["inputs"]["points"].append(input["inputs"]["points"])
        if i == max-1:
            print("Batch Size!")
            break
    prep_input = preprocessor(model_inputs)
    prep_input["inputs"]["voxels"]["voxels"] = prep_input["inputs"]["voxels"]["voxels"].cuda()
    return prep_input

In [9]:
prep_input = prepare_input({"points": pcl}, batch_size=1)

Batch Size!
Preprocessing took 4.820346832275391 ms


In [6]:
model = inferencer.model


In [12]:
a = [1,2,3]
b = a
b[0] = 68
a

[68, 2, 3]

In [15]:
import time 
import torch

torch.manual_seed(42)
s = time.time()

prep_input = prepare_input({"points": pcl}, batch_size=1) # Preprocess
res = model(prep_input["inputs"],prep_input["data_samples"], mode="tensor") # Forward
preds  = model.bbox_head.predict_by_feat(*res) # Postprocess

print("Time: {} ms".format((time.time()-s)*1000))
print(preds)

Batch Size!
Preprocessing took 4.385232925415039 ms
Voxel Encoder took 3.142118453979492 ms
Middle Encoder/Imagify took 8.364200592041016 ms
Backbone + FPN took 3.4232139587402344 ms
Bbox Head forward took 0.274658203125 ms

Time: 66.46871566772461 ms
[<InstanceData(

    META INFORMATION

    DATA FIELDS
    bboxes_3d: LiDARInstance3DBoxes(
            tensor([[3.8503e+01, 2.7783e-01, 4.6321e-01, 4.8079e+00, 1.8755e+00, 1.2150e+00,
                 2.7990e-02]], device='cuda:0'))
    labels_3d: tensor([0], device='cuda:0')
    scores_3d: tensor([0.9256], device='cuda:0')
) at 0x7f8f8da84d60>]


In [128]:
inf_res = inferencer(inputs, show=False,pred_score_thr=0.3, batch_size=4)
inf_res

Preprocessing took 10.436296463012695 ms
Voxel Encoder took 27009.531021118164 ms
Middle Encoder/Imagify took 5892.206907272339 ms
Backbone + FPN took 9514.796257019043 ms
Bbox Head pred took 3506.218433380127 ms



{'predictions': [{'labels_3d': [0],
   'scores_3d': [0.9243199229240417],
   'bboxes_3d': [[38.9719123840332,
     0.44309741258621216,
     0.4683852791786194,
     4.806761741638184,
     1.8732784986495972,
     1.2071927785873413,
     0.15700256824493408]],
   'box_type_3d': 'LiDAR'}],
 'visualization': []}

Convert Manually the tensor to predictions

In [26]:
from mmdet3d.models.dense_heads.anchor3d_head import Anchor3DHead

# bbox_head=dict(
#     type='Anchor3DHead',
#     num_classes=3,
#     in_channels=384,
#     feat_channels=384,
#     use_direction_classifier=True,
#     assign_per_class=True,
#     anchor_generator=dict(
#         type='AlignedAnchor3DRangeGenerator',
#         ranges=[
#             [0, -39.68, -0.6, 69.12, 39.68, -0.6],
#             [0, -39.68, -0.6, 69.12, 39.68, -0.6],
#             [0, -39.68, -1.78, 69.12, 39.68, -1.78],
#         ],
#         sizes=[[0.8, 0.6, 1.73], [1.76, 0.6, 1.73], [3.9, 1.6, 1.56]],
#         rotations=[0, 1.57],
#         reshape_out=False),
#     diff_rad_by_sin=True,
#     bbox_coder=dict(type='DeltaXYZWLHRBBoxCoder')
# )
# model.bbox_head
head = Anchor3DHead(
    num_classes=1,
    in_channels=384,
    feat_channels=384,
    use_direction_classifier=True,
    assign_per_class=True,
    anchor_generator=dict(
                type='AlignedAnchor3DRangeGenerator',
                ranges=[[0, -39.68, -1.78, 69.12, 39.68, -1.78]],
                sizes=[[3.9, 1.6, 1.56]],
                rotations=[0, 1.57],
                reshape_out=True),
    diff_rad_by_sin=True,
    bbox_coder=dict(type='DeltaXYZWLHRBBoxCoder'),
    test_cfg=model.test_cfg
    )
# preds  = head.predict_by_feat(*res)
preds  = model.bbox_head.predict_by_feat(*res)
preds

/home/riccardo/LidarObjDetection/mmdetection3d_dona/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:94: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(


[<InstanceData(
 
     META INFORMATION
 
     DATA FIELDS
     bboxes_3d: LiDARInstance3DBoxes(
             tensor([[3.8503e+01, 2.7785e-01, 4.6314e-01, 4.8079e+00, 1.8755e+00, 1.2150e+00,
                  2.7934e-02]], device='cuda:0'))
     scores_3d: tensor([0.9255], device='cuda:0')
     labels_3d: tensor([0], device='cuda:0')
 ) at 0x7f3a9b63fa90>,
 <InstanceData(
 
     META INFORMATION
 
     DATA FIELDS
     bboxes_3d: LiDARInstance3DBoxes(
             tensor([[39.3502,  0.1765,  0.4314,  4.7754,  1.8597,  1.1957,  0.1487]],
                device='cuda:0'))
     scores_3d: tensor([0.9433], device='cuda:0')
     labels_3d: tensor([0], device='cuda:0')
 ) at 0x7f3a9b63f850>]

In [126]:
preds[0]["bboxes_3d"],pred_res[0]["bboxes_3d"],inf_res["predictions"][0]["bboxes_3d"]

(LiDARInstance3DBoxes(
     tensor([[1.3809, 0.3621, 0.8436, 5.5223, 2.1287, 1.3461, 0.2666]],
        device='cuda:0')),
 LiDARInstance3DBoxes(
     tensor([[38.9716,  0.4431,  0.4685,  4.8068,  1.8733,  1.2072,  0.1570]],
        device='cuda:0')),
 [[38.97159194946289,
   0.44305962324142456,
   0.4684826135635376,
   4.80681848526001,
   1.8733068704605103,
   1.2071855068206787,
   0.15700531005859375]])

# Model Analysis

In [3]:
model = inferencer.model
pcl_path = "../data/cielo_falcon/training/velodyne/"
model_path = "end2end.onnx"

In [4]:

def visualize_preds(preds,inputs,thr=0.3):
    def create_oriented_bounding_box(x, y, z, dx, dy, dz, yaw):
        box = o3d.geometry.OrientedBoundingBox()
        box.center = [x, y, z]
        box.extent = [dx , dy, dz ]
        box.R = o3d.geometry.get_rotation_matrix_from_xyz([0, 0, yaw])
        box.color=[1,0,0]
        return box
    for i in range(len(preds)):
        # Create an Open3D point cloud object
        point_cloud = o3d.geometry.PointCloud()
        point_cloud.points = o3d.utility.Vector3dVector(inputs["points"][i][:,:3])
        point_cloud.colors = o3d.utility.Vector3dVector(np.zeros(inputs["points"][i][:,:3].shape))  # Set all point colors to black
        # x, y, z, dx, dy, dz, yaw = preds[i].bboxes_3d.cpu().numpy().flatten()
        p = preds[i].bboxes_3d.cpu().numpy()
        sc = preds[i].scores_3d.cpu().numpy()
        geometries_to_draw = [point_cloud]
        for j in range(p.shape[0]):
            if sc[i] > thr:
                x, y, z, dx, dy, dz, yaw =p[j,:].flatten()
                box = create_oriented_bounding_box(x, y, z, dx, dy, dz, yaw)
                geometries_to_draw.append(box)
        o3d.visualization.draw_geometries(geometries_to_draw)

def prepare_input(inputs,batch_size=1, device="cuda",data_samples=False):
    model_inputs = {}
    model_inputs["inputs"] = {}
    model_inputs["inputs"]["points"] = []
    if data_samples:
        model_inputs["data_samples"] = []
    max = batch_size

    files = []
    if not os.path.isdir(inputs["points"]):
        files.append(inputs["points"])
    else:
        files = [inputs["points"] + s for s in os.listdir(inputs["points"])]
    for i,sample_path in enumerate(files):
        inp = {}
        inp["lidar_points"] = {}
        inp["lidar_points"]["lidar_path"] = sample_path


        loader = LoadPointsFromFile(coord_type='LIDAR',load_dim=4,use_dim=4)
        packer = Pack3DDetInputs(keys=['points'])

        voxel_size = [0.16, 0.16, 4]
        preprocessor = Det3DDataPreprocessor(
            voxel=True,
            voxel_layer=dict(
                max_num_points=32,  # max_points_per_voxel
                point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1],
                voxel_size=voxel_size,
                max_voxels=(16000, 40000)))

        input = loader(inp)
        input = packer(input)
        # input["inputs"]["points"] = input["inputs"]["points"].cuda()
        model_inputs["inputs"]["points"].append(input["inputs"]["points"])
        if data_samples:
            model_inputs["data_samples"].append(input["data_samples"])
        if i == max-1:
            # print("Batch Size!")
            break
    prep_input = preprocessor(model_inputs)
    if device == "cuda":
        prep_input["inputs"]["voxels"]["voxels"] = prep_input["inputs"]["voxels"]["voxels"].cuda()
    return prep_input

In [ ]:
            # print("Batch Size!")
            break
    prep_input = preprocessor(model_inputs)
    if device == "cuda":
        prep_input["inputs"]["voxels"]["voxels"] = prep_input["inputs"]["voxels"]["voxels"].cuda()
    return prep_input

### Visualization

In [8]:
pcl_path = "../data/cielo_falcon/training/velodyne/"
val_path = "../data/cielo_falcon/ImageSets/val.txt"

with open(val_path, 'r') as file:
    for line in file:
        print(line)
        inputs = prepare_input({"points": os.path.join(pcl_path,line[:-1]+".bin")}, batch_size=1)["inputs"]
        preds = model(inputs,mode="predict")
        visualize_preds(preds,inputs,thr=0.3)

000224

000148

000120

000181

000188



KeyboardInterrupt: 

### Random

In [32]:
import torch
import os
import time
from mmdet3d.datasets.transforms.loading import (LoadAnnotations3D,
                                                LoadPointsFromFile)
from mmdet3d.models.data_preprocessors.data_preprocessor import \
    Det3DDataPreprocessor
from mmdet3d.datasets.transforms.formating import Pack3DDetInputs

pcl_path = "../data/DonaSet/training/velodyne/"
s = time.time()

inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]
# Evaluate on validation data
model.eval()
with torch.no_grad():
    val_output = model(inputs,mode="tensor")
    print("Time: {} ms".format((time.time()-s)*1000))


Batch Size!
Time: 26.506423950195312 ms


In [31]:
import onnx

onnx_model_path = "end2end.onnx"
m = onnx.load(onnx_model_path)
opset_version = m.opset_import[0].version if m.opset_import else None

print("ONNX Model Operator Set Version:", opset_version)

ONNX Model Operator Set Version: 11


In [10]:
import onnxruntime

ort_version = onnxruntime.__version__
print("ONNX Runtime Version:", ort_version)

print( onnxruntime.get_device()  )

ONNX Runtime Version: 1.15.1
GPU


In [80]:

import onnx
import onnxruntime as ort
import numpy as np

model_path = 'end2end.onnx'
onnx_model = onnx.load(model_path)
onnx.checker.check_model(onnx_model)

ort_sess = ort.InferenceSession(model_path,providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider'])


2024-02-22 12:29:06.650643080 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-02-22 12:29:06 WARNING] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
2024-02-22 12:29:06.650668472 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-02-22 12:29:06 WARNING] onnx2trt_utils.cpp:400: One or more weights outside the range of INT32 was clamped
2024-02-22 12:29:06.729307543 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-02-22 12:29:06 WARNING] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
2024-02-22 12:29:06.729332463 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-02-22 12:29:06 WARNING] onnx2trt_utils.cpp:400: One or more weights outside the range of INT32 was clamped


In [81]:
s0 = time.time()
inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]
s05 = time.time()
onnx_inputs = {'voxels': inputs["voxels"]["voxels"].cpu().numpy(), 'coors':inputs["voxels"]["coors"].cpu().numpy(),'num_points':inputs["voxels"]["num_points"].cpu().numpy()}
s1 = time.time()
outputs = ort_sess.run(None, onnx_inputs)
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*outputs)
s3 = time.time()

print("Preprocessing took: {} ms".format((s05-s0)*1000))
print("Cpu conv took: {} ms".format((s1-s05)*1000))
print("Inference took: {} ms".format((s2-s1)*1000))
print("Postprocessing took: {} ms".format((s3-s2)*1000))
print("Total: {} ms".format((s3-s0)*1000))

Batch Size!
Preprocessing took 2.275705337524414 ms
Preprocessing took: 4.957437515258789 ms
Cpu conv took: 1.6481876373291016 ms
Inference took: 18684.181928634644 ms
Postprocessing took: 3.123760223388672 ms
Total: 18693.91131401062 ms


In [82]:
inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]
s05 = time.time()
onnx_inputs = {'voxels': inputs["voxels"]["voxels"].cpu().numpy(), 'coors':inputs["voxels"]["coors"].cpu().numpy(),'num_points':inputs["voxels"]["num_points"].cpu().numpy()}
s1 = time.time()


Batch Size!
Preprocessing took 2.525806427001953 ms


## Inference time comparison

### Pytorch CPU

In [7]:
model.cpu()
inputs = prepare_input({"points": pcl_path}, batch_size=1, device = "cpu")["inputs"]

s1 = time.time()
val_output = model(inputs,mode="tensor")
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*val_output)

print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s1)*1000))

Batch Size!
Inf: 289.40629959106445 ms
Post: 5.438566207885742 ms
Total: 294.905424118042 ms


### Pytorch GPU

In [14]:
model.cuda()
inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]

s1 = time.time()
val_output = model(inputs,mode="tensor")
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*val_output)
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s1)*1000))

Inf: 13.108491897583008 ms
Post: 35.6135368347168 ms
Total: 48.75373840332031 ms


In [29]:
preds[0].bboxes_3d

LiDARInstance3DBoxes(
    tensor([[ 32.2025,  -4.3633,   0.6378,   4.0179,   1.6634,   1.6580,  -0.1687],
        [ 44.4085,   4.6570,   0.6888,   3.9835,   1.6474,   1.6372,   0.1977],
        [ 40.4617,   1.3992,   0.6364,   3.9788,   1.6458,   1.6358,  -0.0796],
        [ 12.4731,  10.0320,   0.6872,   4.4271,   1.8337,   1.8257,   1.1164],
        [ 27.3184, -13.2922,   0.6940,   3.9669,   1.6408,   1.6315,  -0.1891]],
       device='cuda:0'))

In [31]:
visualize_preds(preds,inputs)

### TensorRT Provider (ONNX Runtime)

In [11]:
ort_sess = ort.InferenceSession(model_path,providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider'])

2024-02-22 15:07:18.276197313 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-02-22 15:07:18 WARNING] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
2024-02-22 15:07:18.276223960 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-02-22 15:07:18 WARNING] onnx2trt_utils.cpp:400: One or more weights outside the range of INT32 was clamped
2024-02-22 15:07:18.357865854 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-02-22 15:07:18 WARNING] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
2024-02-22 15:07:18.357891501 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-02-22 15:07:18 WARNING] onnx2trt_utils.cpp:400: One or more weights outside the range of INT32 was clamped


In [15]:
inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]
onnx_inputs = {'voxels': inputs["voxels"]["voxels"].cpu().numpy(), 'coors':inputs["voxels"]["coors"].cpu().numpy(),'num_points':inputs["voxels"]["num_points"].cpu().numpy()}

s1 = time.time()
outputs = ort_sess.run(None, onnx_inputs)
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*outputs)
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s1)*1000))


Batch Size!
Inf: 23.366451263427734 ms
Post: 4.77290153503418 ms
Total: 28.179168701171875 ms


### CUDA Provider  (ONNX Runtime)

In [16]:
ort_sess = ort.InferenceSession(model_path,providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])

In [20]:
inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]
onnx_inputs = {'voxels': inputs["voxels"]["voxels"].cpu().numpy(), 'coors':inputs["voxels"]["coors"].cpu().numpy(),'num_points':inputs["voxels"]["num_points"].cpu().numpy()}

s1 = time.time()
outputs = ort_sess.run(None, onnx_inputs)
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*outputs)
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s1)*1000))

Batch Size!
Inf: 36.96608543395996 ms
Post: 6.192684173583984 ms
Total: 43.22385787963867 ms


### CPU Provider  (ONNX Runtime)

In [21]:
ort_sess = ort.InferenceSession(model_path,providers=[ 'CPUExecutionProvider'])

In [25]:
inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]
onnx_inputs = {'voxels': inputs["voxels"]["voxels"].cpu().numpy(), 'coors':inputs["voxels"]["coors"].cpu().numpy(),'num_points':inputs["voxels"]["num_points"].cpu().numpy()}

s1 = time.time()
outputs = ort_sess.run(None, onnx_inputs)
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*outputs)
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s1)*1000))

Batch Size!
Inf: 194.2272186279297 ms
Post: 48.16293716430664 ms
Total: 242.44952201843262 ms


###  TensorRT

FP32

In [28]:
from mmdeploy.apis.inference import inference_model
from mmdeploy.apis.inference import get_model

model_cfg = "../configs/pointpillars/pointpillars_hv_secfpn_8xb6-160e_donaset-3d-car.py"
deploy_cfg = "/root/workspace/mmdeploy/configs/mmdet3d/voxel-detection/voxel-detection_tensorrt_dynamic-kitti-32x4.py"
backend_files = "end2end.engine"
img = "../data/DonaSet/training/velodyne/000000.bin"
device = "cuda"

tensorrt_model = get_model(model_cfg,deploy_cfg,[backend_files],img,device)

In [31]:

times = 100
sum_pre = 0
sum_inf = 0
sum_post = 0
for i in range(times):
    s0 = time.time()
    inputs = prepare_input({"points": pcl_path}, batch_size=1,data_samples=False)["inputs"]
    s1=time.time()
    res = tensorrt_model(inputs, mode="tensor")
    s2 =time.time()
    preds  = model.bbox_head.predict_by_feat(res["cls_score"],res["bbox_pred"],res["dir_cls_pred"])
    sum_post += time.time() -s2
    sum_inf += s2-s1
    sum_pre += s1 - s0
print("Inf: {} ms".format((sum_pre/times)*1000))
print("Inf: {} ms".format((sum_inf/times)*1000))
print("Post: {} ms".format((sum_post/times)*1000))
print("Total: {} ms".format(((sum_inf+sum_post+sum_pre)/times)*1000))

Inf: 4.0476155281066895 ms
Inf: 1.1346888542175293 ms
Post: 23.787331581115723 ms
Total: 28.96963596343994 ms


In [32]:
visualize_preds(preds,inputs)

FP16

In [33]:
from mmdeploy.apis.inference import inference_model
from mmdeploy.apis.inference import get_model

model_cfg = "../configs/pointpillars/pointpillars_hv_secfpn_8xb6-160e_donaset-3d-car.py"
deploy_cfg = "/root/workspace/mmdeploy/configs/mmdet3d/voxel-detection/voxel-detection_tensorrt_dynamic-kitti-32x4.py"
backend_files = "end2end_fp16.engine"
img = "../data/DonaSet/training/velodyne/000000.bin"
device = "cuda"

tensorrt_model = get_model(model_cfg,deploy_cfg,[backend_files],img,device)

In [34]:

times = 100
sum_pre = 0
sum_inf = 0
sum_post = 0
for i in range(times):
    s0 = time.time()
    inputs = prepare_input({"points": pcl_path}, batch_size=1,data_samples=False)["inputs"]
    s1=time.time()
    res = tensorrt_model(inputs, mode="tensor")
    s2 =time.time()
    preds  = model.bbox_head.predict_by_feat(res["cls_score"],res["bbox_pred"],res["dir_cls_pred"])
    sum_post += time.time() -s2
    sum_inf += s2-s1
    sum_pre += s1 - s0
print("Inf: {} ms".format((sum_pre/times)*1000))
print("Inf: {} ms".format((sum_inf/times)*1000))
print("Post: {} ms".format((sum_post/times)*1000))
print("Total: {} ms".format(((sum_inf+sum_post+sum_pre)/times)*1000))

Inf: 4.298429489135742 ms
Inf: 1.263444423675537 ms
Post: 14.704809188842773 ms
Total: 20.266683101654053 ms


In [21]:
visualize_preds(preds,inputs)